{/* cspell:ignore Kipu, DCQO, QUBO, HUBO, counterdiabatic, Iskay, bitflips */}

# Iskay Quantum Optimizer - A Qiskit Function by Kipu Quantum

<LegacyContent>
<Admonition type="note" title="Notes">
* This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.]( https://quantum.cloud.ibm.com/docs/guides/kipu-optimization)
* Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note" title="Notes">
* This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/kipu-optimization)
* Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>
</CloudContent>

## Overview

With the Iskay Quantum Optimizer by Kipu Quantum, you can tackle complex optimization problems using IBM&reg; quantum computers. This solver leverages Kipu's cutting-edge [bf-DCQO algorithm](https://doi.org/10.48550/arXiv.2409.04477) requiring only the objective function as input to automatically deliver problem solutions. It can handle optimization problems involving up to 156 qubits, enabling the use of all qubits of the IBM quantum devices. The Optimizer uses a 1-to-1 mapping between classical variables and qubits, which allows you to tackle optimization problems with up to 156 binary variables.

The Optimizer enables the solving of unconstrained binary optimization problems. In addition to the commonly used QUBO (Quadratic Unconstrained Binary Optimization) formulation, it also supports higher-order (HUBO) optimization problems. The solver utilizes a non-variational quantum algorithm, performing most of the computation on quantum devices.

The following provides more details about the used algorithm and a brief guide on how to use the function, as well as benchmarking results on various problem instances of different sizes and complexities.

## Description

The Optimizer is a ready-to-use implementation of cutting-edge quantum optimization algorithms. It solves optimization problems by running highly-compressed quantum circuits on quantum hardware. This compression is achieved by introducing counterdiabatic terms into the underlying time evolution of the quantum system. The algorithm executes several iterations of hardware runs to obtain the final solutions and combines it with post-processing. These steps are seamlessly integrated into the Optimizer's workflow and are executed automatically.

### How does the Quantum Optimizer work?
This section outlines the basics of the implemented bf-DCQO algorithm. An introduction to the algorithm can also be found on the [Qiskit YouTube channel.](https://www.youtube.com/watch?v=33QmsXhIlpU&t=1223s)

The algorithm is based on the time evolution of a quantum system which is transformed over time, where the problem solution is encoded in the ground state of the quantum system at the end of the evolution. According to the [adiabatic theorem](https://en.wikipedia.org/wiki/Adiabatic_theorem), this evolution has to be slow to ensure the system remains in its ground state. Digitizing this evolution is the basis of digitized quantum adiabatic computation (DQA) and the infamous QAOA algorithm. However, the required slow evolution is not feasible for increasing problem sizes since it results in an increasing circuit depth. By using counterdiabatic protocols, you can suppress unwanted excitations occurring during short evolution times while remaining in the ground state. Here, digitizing this shorter evolution time results in quantum circuits with shorter depth and fewer entangling gates.

The circuits of the bf-DCQO algorithms typically use up to ten times fewer entangling gates than DQA, and three to four times fewer entangling gates than standard QAOA implementations. Because of the smaller number of gates, fewer errors occur during the circuit execution on hardware. Hence, the optimizer does not require using techniques like error suppression or error mitigation. Implementing them in future versions can enhance the solution quality even further.

Although the bf-DCQO algorithm uses iterations, it is non-variational. After each iteration of the algorithm, the distribution of states is measured. The obtained distribution is used to calculate a so-called bias-field. The bias-field allows starting the next iteration from an energy state near the previously found solution. In this way, the algorithm moves with each iteration to solutions of lower energy. Typically, approximately ten iterations are sufficient to converge to a solution, in total requiring a much lower number of iterations than variational algorithms, which is on the order of  approximately 100 iterations.

The optimizer combines the bf-DCQO algorithm with classical post-processing. After measuring the distribution of states, a local search is performed. During the local search, the bits of the measured solution are randomly flipped. After the flip, the energy of the new bitstring is evaluated. If the energy is lower, the bitstring is kept as the new solution. The local search only scales linearly with the number of qubits; hence, it is computationally cheap. Since the post-processing corrects local bitflips, it compensates for bit-flip errors that often are the result of hardware imperfections and readout errors.

### Workflow

A schematic of the workflow of the Quantum Optimizer follows.

![Workflow](/images/guides/kipu-optimization/workflow.svg "Workflow of the Quantum Optimizer")

By using the Quantum Optimizer, solving an optimization problem on quantum hardware can be reduced to
* Formulate the objective function of the problem
* Access the Optimizer via Qiskit Functions
* Run the Optimizer and collect the result

## Benchmarks

The benchmark metrics below show that the Optimizer effectively addresses problems involving up to 156 qubits and offer a general overview of the optimizer's accuracy and scalability across different problem types. Note that actual performance metrics may vary depending on specific problem characteristics, such as the number of variables, the density and locality of terms in the objective function, and the polynomial order.

The following table includes the approximation ratio (AR), a metric defined as follows:
$$
AR = \frac{C^{*} - C_\textrm{max}}{C_{\textrm{min}} - C_{\textrm{max}}},
$$
where $C$ is the objective function, $C_{\textrm{min}}$, $C_{\textrm{max}}$ are its minimum and maximum values, and $C^{*}$ is the cost of the best solution found, respectively. Therefore, AR=100\% means that the ground state of the problem has been obtained.


| Example           | Number of qubits | Approximation Ratio | Total time (s) | Runtime usage (s) | Total Number of shots | Number of iterations |
| ----------------- | :--------------: | :------: | :------------: | :---------------: | :-------------------: | :------------------: |
| Unweighted MaxCut |        28        |   100%   |      180       |        30         |          30k          |          5           |
| Unweighted MaxCut |        30        |   100%   |      180       |        30         |          30k          |          5           |
| Unweighted MaxCut |        32        |   100%   |      180       |        30         |          30k          |          5           |
| Unweighted MaxCut |        80        |   100%   |      480       |        60         |          90k          |          9           |
| Unweighted MaxCut |       100        |   100%   |      330       |        60         |          60k          |          6           |
| Unweighted MaxCut |       120        |   100%   |      370       |        60         |          60k          |          6           |
| HUBO 1            |       156        |   100%   |      600       |        70         |         100k          |          10          |
| HUBO 2            |       156        |   100%   |      600       |        70         |         100k          |          10          |

- The MaxCut instances with 28, 30, and 32 qubits were run on ibm_sherbrooke. Instances with 80, 100, and 120 were run on ibm_marrakesh.
- The HUBO instances were also run on ibm_marrakesh.

All the benchmark instances are accessible on GitHub (see [Kipu benchmark instances](https://github.com/Kipu-Quantum-GmbH/benchmark-instances)). An example to run these instances can be found in [Example 3: Benchmark instances](#example-3-benchmark-instances).

## Inputs and outputs

### Input
See the following table for all input parameters the Quantum Optimizer accepts.  The subsequent **Options** section goes into more details about the available `options`.

<LegacyContent>
| Name         | Type               | Description                                                                                                                                                                            | Required | Default                                                                                       | Example                                                                                                              |
| ------------ | ------------------ | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------- | --------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| problem      | `Dict[str, float]` | The coefficients of the optimization problem formulated as QUBO/HUBO or spin format. For more information on the problem specification, see [Accepted problem formats](#accepted-problem-formats)                                       | Yes      | N/A                                                                                           | `{"()": -21.0, "(0, 4)": 0.5,"(0, 2)": 0.5,"(0, 1)": 0.5,"(1, 3)": 0.5}`                                             |
| problem_type | `str`              | Specify whether the problem coefficients are in binary (QUBO/HUBO) or spin format. The two possibilities are `"spin"` or `"binary"`                                                   | Yes      | N/A                                                                                           | `"spin"`                                                                                                             |
| backend_name | `str`              | Name of the backend to make the query                                                                                                                                                  | Yes       | N/A                                                      | `"ibm_fez"`                                                                                                          |
| instance     | `str`              | The hub/group/project to use                                                                                                                                                           | No       | A Premium access instance is randomly chosen if your account has access to multiple instances. | "hub1/group1/project1"                                                                                               |
| options      | `Dict[str, Any]`   | Options to handle the hardware submission, such as number of shots. For further details on the options configuration, see the [Options](#options) section | No       | To see the default values of the options configuration see the [Options](#options) section                                                         | `{"shots": 5000, "num_iterations": 3, "use_session": True, "seed_transpiler": 42}` |
</LegacyContent>
<CloudContent>
| Name         | Type               | Description                                                                                                                                                                            | Required | Default                                                                                       | Example                                                                                                              |
| ------------ | ------------------ | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------- | --------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| problem      | `Dict[str, float]` | The coefficients of the optimization problem formulated as QUBO/HUBO or spin format. For more information on the problem specification, see [Accepted problem formats](#accepted-problem-formats)                                       | Yes      | N/A                                                                                           | `{"()": -21.0, "(0, 4)": 0.5,"(0, 2)": 0.5,"(0, 1)": 0.5,"(1, 3)": 0.5}`                                             |
| problem_type | `str`              | Specify whether the problem coefficients are in binary (QUBO/HUBO) or spin format. The two possibilities are `"spin"` or `"binary"`                                                   | Yes      | N/A                                                                                           | `"spin"`                                                                                                             |
| backend_name | `str`              | Name of the backend to make the query                                                                                                                                                  | Yes       | N/A                                                      | `"ibm_fez"`                                                                                                          |
| instance     | `str`              | The cloud resource name of the instance to use                                                                                                                                                           | No       | A Premium access instance is randomly chosen if your account has access to multiple instances. | "CRN"                                                                                               |
| options      | `Dict[str, Any]`   | Options to handle the hardware submission, such as number of shots. For further details on the options configuration, see the [Options](#options) section | No       | To see the default values of the options configuration see the [Options](#options) section                                                         | `{"shots": 5000, "num_iterations": 3, "use_session": True, "seed_transpiler": 42}` |
</CloudContent>

 #### Accepted problem formats
The `problem` and `problem_type` arguments encode an optimization problem of the form

$$
\begin{align}
\min_{(x_1, x_2, \ldots, x_n) \in D} C(x_1, x_2, \ldots, x_n) \nonumber
\end{align}
$$
where

$$
C(x_1, ... , x_n) = a + \sum_{i} b_i x_i + \sum_{i, j} c_{i, j} x_i x_j + ... + \sum_{k_1, ..., k_m} g_{k_1, ..., k_m} x_{k_1} ... x_{k_m}
$$

- By choosing `problem_type = "binary"`, you specify that the cost function is in `binary` format, which means that $D = \{0,  1\}^{n}$, as in, the cost function is written in QUBO/HUBO formulation.
- On the other hand, by choosing `problem_type = "spin"`, the cost function is written in Ising formulation, where $D = \{-1, 1\}^{n}$.

The coefficients of the problem should be encoded in a dictionary as follows:
$$
\begin{align} \nonumber
&\texttt{\{} \\ \nonumber
&\texttt{"()"}&: \quad &a, \\ \nonumber
&\texttt{"(i,)"}&: \quad &b_i, \\ \nonumber
&\texttt{"(i, j)"}&: \quad &c_{i, j}, \\ \nonumber
&\quad  \vdots \\ \nonumber
&\texttt{"(} k_1, ..., k_m  \texttt{)"} &: \quad &g_{k_1, ..., k_m}, \\ \nonumber
&\texttt{\}}
\end{align}
$$

- Please note that the keys of the dictionary must be strings containing a valid tuple of non-repeating integers.

#### Options

The hyper-parameters of the algorithm are set automatically. However, you can fine-tune the algorithm by exposing parameters to the user. For example, you can select the number of iterations of the algorithm.

| Name                 | Type        | Description                                                                                                                                                                                                    | Required | Default |
| -------------------- | ----------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------- | ------- |
| shots                | `int`       | The number of shots per iteration to execute on hardware.                                                                                                                                                       | No       | 10000   |
| num_iterations       | `int`       | The total number of Bias Field iterations performed by the solver.                                                                                                                                              | No       | 10      |
| use_session          | `bool`      | Whether to run the solver within an IBM session.                                                                                                                                                                | No       | True    |
| seed_transpiler      | `int`       | Optional parameter to set a specific seed to the transpiler, ensures that the same circuit is submitted to hardware.                                                                                           | No       | None    |
| direct_qubit_mapping | `bool`      | Optional parameter to indicate that virtual and physical qubits should be addressed by the same index. This option should only be turned on when the connectivity of the problem matches that of the hardware. | No       | False   |
| job_tags             | `List[str]` | Optional parameter to include additional tags to the quantum jobs executed by the optimizer. Note, that any job submitted with the Iskay quantum optimizer will contain the tags 'Kipu_Quantum' and 'BF-DCQO'. | No       | None    |

### Output

| Name   | Type               | Description                                                           | Example                                                                |
| ------ | ------------------ | ----------------------------------------------------------------------| ---------------------------------------------------------------------- |
| result | `Dict[str, float]` | Solution and metadata listed under "Result dictionary contents".       |   `{'solution': {'bitstring': '011', 'cost': -23.5, 'mapped_solution': {'0': 0, '1': 1, '2': 1} , 'mapping': {0: 0, 2: 1, 1: 2}, 'problem type': 'spin', 'seed': 42}`      |

### Result dictionary contents

| Name         | Type               | Description                                                                                                             |
| ------------ | :------------------: | ----------------------------------------------------------------------------------------------------------------------- |
| solution     | `Dict` | The solution to the optimization problem in the format it was formulated. This corresponds to a dictionary with items `var_label: var_value` where each `var_label` is a variable in the optimization problem, and `var_value` is its corresponding value (spin or binary) in the optimal solution.|
| solution_info      | `Dict`   | The mapping from the bitstring bit to the equivalent variable in the problem                                            |
| prob_type | `str`             | Whether the problem is formulated in binary or spin variables                                                           |

#### solution_info dictionary
This is a dictionary containing the details of the solution found by the optimizer. It contains the following items:
  - **"bitstring"**: The optimal bitstring measured.
  - **"cost"**: The cost value of the solution.
  - **"mapping"**: The mapping from the bitstring bit to the equivalent variable in the problem.
  - **"seed_transpiler"**: Seed to the transpiler, can be used to ensure the same exact quantum circuit is submitted to hardware. If a seed is not provided, a seed search is run to optimize circuit depth.


Note that the `solution` dictionary is obtained from the solution bitstring, using the `mapping` object for indexing the variables. When `problem_type=spin` we use the assignment $1 \rightarrow -1, \quad 0 \rightarrow 1$.

## Get started
<LegacyContent>
Authenticate using your [IBM Quantum&reg; Platform API token](http://quantum.ibm.com/), and select the Qiskit Function as follows:
</LegacyContent>
<CloudContent>
Authenticate using your API key, found on the [IBM Quantum Platform dashboard](http://quantum.cloud.ibm.com/), and select the Qiskit Function as follows:
</CloudContent>

In [ ]:
# ruff: noqa: F821

<Admonition type="note">
The following code assumes that you have saved your credentials. If you have not, follow the instructions in <LegacyContent>[Set up an IBM Quantum channel](/guides/setup-channel#iqp)</LegacyContent><CloudContent>[Set up your IBM Cloud account](/guides/cloud-setup#cloud-untrusted)</CloudContent> to authenticate with your API key.
</Admonition>

In [ ]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()

# Access Function
optimizer = catalog.load("kipu-quantum/iskay-quantum-optimizer")

## Example 1: Simple cost function

Consider the cost function in spin formulation:
$$
C(x_0, x_1, x_2, x_3, x_4) = 1 + 1.5x_0 + 2x_1 + 1.3x_2 + 2.5x_0x_3 + 3.5x_1x_4 + 4x_0x_1x_2
$$

where $(x_0, ..., x_4) \in \{-1, 1\}^5$.

The solution to this simple cost function is
$$
(x_0, x_1, x_2, x_3, x_4) = (-1, -1, -1, 1, 1)
$$
with minimum value $C^{*} = -6$

### 1. Create the objective function

We start by creating a dictionary with the coefficients of the objective function as follows:

In [ ]:
objective_func = {
    "()": 1,
    "(0,)": 1.5,
    "(1,)": 2,
    "(2,)": 1.3,
    "(0, 3)": 2.5,
    "(1, 4)": 3.5,
    "(0, 1, 2)": 4,
}

### 2. Run the Optimizer

We solve the problem by running the optimizer. Since $(x_0, ..., x_4) \in \{-1, 1\}^5$ we must set `problem_type=spin`.

In [ ]:
# Setup options to run the optimizer
options = {"shots": 5000, "num_iterations": 5, "use_session": True}

arguments = {
    "problem": objective_func,
    "problem_type": "spin",
    "instance": instance,
    "backend_name": backend_name,  # such as "ibm_fez"
    "options": options,
}

job = optimizer.run(**arguments)

### 3. Retrieve the result

The solution of the optimization problem is provided directly from the optimizer.

In [ ]:
print(job.result())

This will show a dictionary of the form:

```
{'solution': {'0': -1, '1': -1, '2': -1, '3': 1, '4': 1},
 'solution_info': {'bitstring': '11100',
  'cost': -13.8,
  'seed_transpiler': 42,
  'mapping': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}},
 'prob_type': 'spin'}
```

Notice that the dictionary `solution` displays the result vector $(x_0, x_1, x_2, x_3, x_4) = (-1, -1, -1, 1, 1)$.

## Example 2: MaxCut

Many graph problems like MaxCut or Maximum independent set are NP-hard problems and ideal candidates for testing quantum algorithms and hardware. This example demonstrates solving the MaxCut problem of a 3-regular graph with the Quantum Optimizer.

To run this example you must install the `networkx` package in addition to the `qiskit-ibm-catalog`. To install it, run the following command:

In [ ]:
# %pip install networkx numpy

### 1. Create the objective function

Start by generating a random 3-regular graph. For this graph, we define the objective function of the MaxCut problem.

In [ ]:
import networkx as nx

# Create a random 3-regular graph
G = nx.random_regular_graph(3, 10, seed=42)


# Create the objective function for MaxCut in Ising formulation
def graph_to_ising_maxcut(G):
    """
    Convert a NetworkX graph to an Ising Hamiltonian for the Max-Cut problem.
    Args:
        G (networkx.Graph): The input graph.
    Returns:
        dict: The objective function of the Ising model
    """
    # Initialize the linear and quadratic coefficients
    objective_func = {}
    # Populate the coefficients
    for i, j in G.edges:
        objective_func[f"({i}, {j})"] = 0.5
    return objective_func


objective_func = graph_to_ising_maxcut(G)

### 2. Run the Optimizer

Solve the problem by running the optimizer.

In [ ]:
options = {"shots": 5000, "num_iterations": 5, "use_session": True}

arguments = {
    "problem": objective_func,
    "problem_type": "spin",
    "instance": instance,
    "backend_name": backend_name,  # such as "ibm_fez"
    "options": options,
}

job = optimizer.run(**arguments)

### 3. Retrieve the result

Retrieve the result and map the solution bitstring back to the original graph nodes.

In [ ]:
print(job.result())

The solution to the Maxcut problem is directly contained in the `solution` sub-dictionary of the result object

In [ ]:
maxcut_solution = job.result()["solution"]

## Example 3: Benchmark instances

The benchmark instances are available on GitHub: [Kipu benchmark instances.](https://github.com/Kipu-Quantum-GmbH/benchmark-instances)

The instances can be loaded using the `pygithub` library. To install it, run the following command:

In [ ]:
# %pip install pygithub

The paths for the benchmark instances are:

**Maxcut:**
- `'maxcut/maxcut_28_nodes.json'`
- `'maxcut/maxcut_30_nodes.json'`
- `'maxcut/maxcut_32_nodes.json'`
- `'maxcut/maxcut_80_nodes.json'`
- `'maxcut/maxcut_100_nodes.json'`
- `'maxcut/maxcut_120_nodes.json'`

**HUBO:**
- `'HUBO/hubo1_marrakesh.json'`
- `'HUBO/hubo2_marrakesh.json'`

To reproduce the performance of the benchmark for the HUBO instances, select the backend `ibm_marrakesh` and set `direct_qubit_mapping` to `True` in the `options` sub-dictionary.

The following example runs the Maxcut instance with 32 nodes.

In [ ]:
from github import Github
import urllib
import json
import ast

repo = "Kipu-Quantum-GmbH/benchmark-instances"
path = "maxcut/maxcut_32_nodes.json"
gh = Github()
repo = gh.get_repo(repo)
branch = "main"
file = repo.get_contents(urllib.parse.quote(path), ref=branch)

# load json file with benchmark problem
problem_json = json.loads(file.decoded_content)

# convert objective function to compatible format
objective_func = {
    key: ast.literal_eval(value) for key, value in problem_json.items()
}


# Setup configuration to run the optimizer
options = {
    "shots": 5_000,
    "num_iterations": 5,
    "use_session": True,
    "direct_qubit_mapping": False,
}

arguments = {
    "problem": objective_func,
    "problem_type": "spin",
    "backend_name": "ibm_brisbane",
    "instance": instance,
    "options": options,
}

job = optimizer.run(**arguments)

result = job.result()

## Use cases

Typical use cases for the Optimization solver are combinatorial optimization problems. You can solve problems from many industries like finance, pharmaceutics, or logistics. Some examples follow.
* Portfolio optimization (QUBO): [scientific publication](https://doi.org/10.1103/PhysRevApplied.22.054037) and [white paper](https://kipu-quantum.com/zope64/kipu_2024/content/e3915/e3916/e4187/White-Paper-2-Financial-modeling-on-quantum-computers-using-digitally-compressed-algorithms-1.pdf)
* Protein folding (HUBO): [scientific publication](https://doi.org/10.1103/PhysRevApplied.20.014024)
* Logistics scheduling (QUBO): [scientific publication](https://doi.org/10.1103/PhysRevApplied.22.064068)
* Network optimization: [webinar](https://www.youtube.com/watch?v=w5SrCIK88No)

If you are interested in tackling a specific use case and develop a dedicated mapping, we can assist you. [Contact us.](https://share-eu1.hsforms.com/2Ff8cgWvTR9ukT_fPoaNhDw2dqpz5)

## Get support

For support, contact support@kipu-quantum.com.

## Next steps

[Request access to the Quantum Optimizer by Kipu Quantum](https://share-eu1.hsforms.com/2Ff8cgWvTR9ukT_fPoaNhDw2dqpz5)

## Additional information

Iskay, like our company name Kipu Quantum, is a Peruvian word. Although we are a startup from Germany, these words come from the native country of one of our co-founders, where the Quipu was one of the very first calculation machines developed by mankind 2000 years BCE.